# Data labeling

For using BERT and comparing results of VADER, I´ll label the dataset in this notebook. Moreover, I will eliminate the end-of-race radios for having a more concise model. If the dataset gets very small, I will raise a flag for extracting more radio messages.